In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('AsuransiKendaraan.csv', delimiter=';')

C:\Users\T480\AppData\Local\Temp\ipykernel_25160\3526871572.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AsuransiKendaraan.csv', delimiter=';')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105555 entries, 0 to 105554
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    105555 non-null  int64  
 1   Date_start_contract   105555 non-null  object 
 2   Date_last_renewal     105555 non-null  object 
 3   Date_next_renewal     105555 non-null  object 
 4   Date_birth            105555 non-null  object 
 5   Date_driving_licence  105555 non-null  object 
 6   Distribution_channel  105555 non-null  object 
 7   Seniority             105555 non-null  int64  
 8   Policies_in_force     105555 non-null  int64  
 9   Max_policies          105555 non-null  int64  
 10  Max_products          105555 non-null  int64  
 11  Lapse                 105555 non-null  int64  
 12  Date_lapse            35147 non-null   object 
 13  Payment               105555 non-null  int64  
 14  Premium               105555 non-null  float64
 15  

In [4]:
df

,ID,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Distribution_channel,Seniority,Policies_in_force,Max_policies,...,Area,Second_driver,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Type_fuel,Length,Weight
0,1,05/11/2015,05/11/2015,05/11/2016,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.00,0,P,NaN,190
1,1,05/11/2015,05/11/2016,05/11/2017,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.00,0,P,NaN,190
2,1,05/11/2015,05/11/2017,05/11/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.00,0,P,NaN,190
3,1,05/11/2015,05/11/2018,05/11/2019,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.00,0,P,NaN,190
4,2,26/09/2017,26/09/2017,26/09/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.00,0,P,NaN,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105550,53498,30/07/2018,30/07/2018,30/07/2019,25/07/1981,14/02/2007,0,1,1,1,...,0,0,2000,110,1997,24320.00,5,D,4.740,1480
105551,53499,16/08/2018,16/08/2018,16/08/2019,08/12/1976,29/11/2017,0,1,1,1,...,0,0,2013,129,1998,30861.97,5,P,4.650,1440
105552,53500,21/11/2018,21/11/2018,21/11/2019,01/04/1974,05/10/2011,0,1,1,1,...,0,0,1999,55,999,7800.00,5,P,3.495,830
105553,53501,21/11/2018,21/11/2018,21/11/2019,15/09/1946,02/02/1982,0,1,1,1,...,0,0,2004,90,1753,16610.00,5,D,4.555,1399


In [5]:
drop_cols_for_new_user = [
    'ID',                       # hanya identifier
    'Date_start_contract',      # drop karena gak ekstrak durasi polis
    'Date_last_renewal',        # sama seperti di atas
    'Date_next_renewal',        # sama seperti di atas
    'Date_driving_licence',     # drop karena gak ekstrak durasi izin mengemudi
    'Date_lapse',               # banyak missing dan sudah ada Lapse
    'Lapse',                    # status polis lapse
    'Max_products',             # kurang relevan
    'Policies_in_force',        # belum ada polis aktif
    'Max_policies',             # belum punya polis
    'Cost_claims_year',         # belum ada klaim
    'N_claims_year',            # belum ada klaim
    'N_claims_history',         # belum ada klaim
    'R_Claims_history',         # belum ada klaim
    'Length'                    # banyak missing, kurang relevan
]

In [6]:
df = df.drop(columns=drop_cols_for_new_user)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105555 entries, 0 to 105554
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Date_birth            105555 non-null  object 
 1   Distribution_channel  105555 non-null  object 
 2   Seniority             105555 non-null  int64  
 3   Payment               105555 non-null  int64  
 4   Premium               105555 non-null  float64
 5   Type_risk             105555 non-null  int64  
 6   Area                  105555 non-null  int64  
 7   Second_driver         105555 non-null  int64  
 8   Year_matriculation    105555 non-null  int64  
 9   Power                 105555 non-null  int64  
 10  Cylinder_capacity     105555 non-null  int64  
 11  Value_vehicle         105555 non-null  float64
 12  N_doors               105555 non-null  int64  
 13  Type_fuel             103791 non-null  object 
 14  Weight                105555 non-null  int64  
dtype

In [7]:
df.rename(columns={
    'Date_birth': 'Tanggal Lahir',
    'Distribution_channel': 'Saluran Distribusi',
    'Seniority': 'Lama Menjadi Pelanggan',
    'Payment': 'Metode Pembayaran',
    'Type_risk': 'Kategori Risiko',
    'Area': 'Wilayah',
    'Second_driver': 'Pengemudi Kedua',
    'Year_matriculation': 'Tahun Pendaftaran Kendaraan',
    'Power': 'Daya Mesin',
    'Cylinder_capacity': 'Kapasitas Silinder',
    'Value_vehicle': 'Nilai Kendaraan',
    'N_doors': 'Jumlah Pintu',
    'Type_fuel': 'Jenis Bahan Bakar',
    'Weight': 'Berat Kendaraan',
    'Premium': 'Premi Asuransi'
}, inplace=True)

# Pindahkan 'Premi Asuransi' dan 'Kategori Risiko' ke paling akhir
cols = list(df.columns)
for col in ['Premi Asuransi', 'Kategori Risiko']:
    cols.remove(col)
cols += ['Premi Asuransi', 'Kategori Risiko']
df = df[cols]

# Cek struktur
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105555 entries, 0 to 105554
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Tanggal Lahir                105555 non-null  object 
 1   Saluran Distribusi           105555 non-null  object 
 2   Lama Menjadi Pelanggan       105555 non-null  int64  
 3   Metode Pembayaran            105555 non-null  int64  
 4   Wilayah                      105555 non-null  int64  
 5   Pengemudi Kedua              105555 non-null  int64  
 6   Tahun Pendaftaran Kendaraan  105555 non-null  int64  
 7   Daya Mesin                   105555 non-null  int64  
 8   Kapasitas Silinder           105555 non-null  int64  
 9   Nilai Kendaraan              105555 non-null  float64
 10  Jumlah Pintu                 105555 non-null  int64  
 11  Jenis Bahan Bakar            103791 non-null  object 
 12  Berat Kendaraan              105555 non-null  int64  
 13 

In [8]:
# Konversi ke Rupiah (pakai USD)
kurs_usd_idr = 15000
df['Premi Asuransi'] = df['Premi Asuransi'] * kurs_usd_idr
df

,Tanggal Lahir,Saluran Distribusi,Lama Menjadi Pelanggan,Metode Pembayaran,Wilayah,Pengemudi Kedua,Tahun Pendaftaran Kendaraan,Daya Mesin,Kapasitas Silinder,Nilai Kendaraan,Jumlah Pintu,Jenis Bahan Bakar,Berat Kendaraan,Premi Asuransi,Kategori Risiko
0,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3337800.0,1
1,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3206700.0,1
2,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3222600.0,1
3,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3254850.0,1
4,15/04/1956,0,4,1,0,0,2004,80,599,7068.00,0,P,190,3205500.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105550,25/07/1981,0,1,0,0,0,2000,110,1997,24320.00,5,D,1480,3956850.0,3
105551,08/12/1976,0,1,0,0,0,2013,129,1998,30861.97,5,P,1440,6284550.0,3
105552,01/04/1974,0,1,1,0,0,1999,55,999,7800.00,5,P,830,8578650.0,3
105553,15/09/1946,0,1,0,0,0,2004,90,1753,16610.00,5,D,1399,5094900.0,2




# Mapping dictionary
- saluran_distribusi_map = {0: 'Agen', 1: 'Online', 2: 'Mitra'}
- metode_pembayaran_map = {0: 'Bulanan', 1: 'Tahunan', 2: 'Sekali Bayar'}
- kategori_risiko_map = {1: 'Rendah', 2: 'Sedang', 3: 'Tinggi'}
- wilayah_map = {0: 'Rendah', 1: 'Sedang', 2: 'Tinggi'}
- pengemudi_kedua_map = {0: 'Tidak Ada', 1: 'Ada'}

In [9]:
df

,Tanggal Lahir,Saluran Distribusi,Lama Menjadi Pelanggan,Metode Pembayaran,Wilayah,Pengemudi Kedua,Tahun Pendaftaran Kendaraan,Daya Mesin,Kapasitas Silinder,Nilai Kendaraan,Jumlah Pintu,Jenis Bahan Bakar,Berat Kendaraan,Premi Asuransi,Kategori Risiko
0,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3337800.0,1
1,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3206700.0,1
2,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3222600.0,1
3,15/04/1956,0,4,0,0,0,2004,80,599,7068.00,0,P,190,3254850.0,1
4,15/04/1956,0,4,1,0,0,2004,80,599,7068.00,0,P,190,3205500.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105550,25/07/1981,0,1,0,0,0,2000,110,1997,24320.00,5,D,1480,3956850.0,3
105551,08/12/1976,0,1,0,0,0,2013,129,1998,30861.97,5,P,1440,6284550.0,3
105552,01/04/1974,0,1,1,0,0,1999,55,999,7800.00,5,P,830,8578650.0,3
105553,15/09/1946,0,1,0,0,0,2004,90,1753,16610.00,5,D,1399,5094900.0,2


In [10]:
# Cek missing values
print(df.isnull().sum())

Tanggal Lahir                     0
Saluran Distribusi                0
Lama Menjadi Pelanggan            0
Metode Pembayaran                 0
Wilayah                           0
Pengemudi Kedua                   0
Tahun Pendaftaran Kendaraan       0
Daya Mesin                        0
Kapasitas Silinder                0
Nilai Kendaraan                   0
Jumlah Pintu                      0
Jenis Bahan Bakar              1764
Berat Kendaraan                   0
Premi Asuransi                    0
Kategori Risiko                   0
dtype: int64


In [11]:
# Imputasi missing value di "Jenis Bahan Bakar"
df['Jenis Bahan Bakar'].fillna(df['Jenis Bahan Bakar'].mode()[0], inplace=True)

C:\Users\T480\AppData\Local\Temp\ipykernel_25160\4259575238.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Jenis Bahan Bakar'].fillna(df['Jenis Bahan Bakar'].mode()[0], inplace=True)


In [12]:
from datetime import datetime

# Ubah ke datetime
df['Tanggal Lahir'] = pd.to_datetime(df['Tanggal Lahir'], errors='coerce')

# Hitung usia
df['Usia'] = datetime.now().year - df['Tanggal Lahir'].dt.year

# Drop kolom aslinya
df.drop('Tanggal Lahir', axis=1, inplace=True)

C:\Users\T480\AppData\Local\Temp\ipykernel_25160\318523431.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Tanggal Lahir'] = pd.to_datetime(df['Tanggal Lahir'], errors='coerce')


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105555 entries, 0 to 105554
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Saluran Distribusi           105555 non-null  object 
 1   Lama Menjadi Pelanggan       105555 non-null  int64  
 2   Metode Pembayaran            105555 non-null  int64  
 3   Wilayah                      105555 non-null  int64  
 4   Pengemudi Kedua              105555 non-null  int64  
 5   Tahun Pendaftaran Kendaraan  105555 non-null  int64  
 6   Daya Mesin                   105555 non-null  int64  
 7   Kapasitas Silinder           105555 non-null  int64  
 8   Nilai Kendaraan              105555 non-null  float64
 9   Jumlah Pintu                 105555 non-null  int64  
 10  Jenis Bahan Bakar            105555 non-null  object 
 11  Berat Kendaraan              105555 non-null  int64  
 12  Premi Asuransi               105555 non-null  float64
 13 

In [14]:
df['Saluran Distribusi'] = df['Saluran Distribusi'].astype(str)
df['Jenis Bahan Bakar'] = df['Jenis Bahan Bakar'].astype(str)

In [15]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
df[['Saluran Distribusi', 'Jenis Bahan Bakar']] = encoder.fit_transform(df[['Saluran Distribusi', 'Jenis Bahan Bakar']])

In [16]:
def map_risk_category(x):
    if x == 1:
        return 0  # risiko rendah
    elif x == 2:
        return 1  # risiko sedang
    else:
        return 2  # risiko tinggi (3 dan 4)

df['Kategori Risiko'] = df['Kategori Risiko'].apply(map_risk_category)

In [17]:
df

,Saluran Distribusi,Lama Menjadi Pelanggan,Metode Pembayaran,Wilayah,Pengemudi Kedua,Tahun Pendaftaran Kendaraan,Daya Mesin,Kapasitas Silinder,Nilai Kendaraan,Jumlah Pintu,Jenis Bahan Bakar,Berat Kendaraan,Premi Asuransi,Kategori Risiko,Usia
0,0.0,4,0,0,0,2004,80,599,7068.00,0,1.0,190,3337800.0,0,69
1,0.0,4,0,0,0,2004,80,599,7068.00,0,1.0,190,3206700.0,0,69
2,0.0,4,0,0,0,2004,80,599,7068.00,0,1.0,190,3222600.0,0,69
3,0.0,4,0,0,0,2004,80,599,7068.00,0,1.0,190,3254850.0,0,69
4,0.0,4,1,0,0,2004,80,599,7068.00,0,1.0,190,3205500.0,0,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105550,0.0,1,0,0,0,2000,110,1997,24320.00,5,0.0,1480,3956850.0,2,44
105551,0.0,1,0,0,0,2013,129,1998,30861.97,5,1.0,1440,6284550.0,2,49
105552,0.0,1,1,0,0,1999,55,999,7800.00,5,1.0,830,8578650.0,2,51
105553,0.0,1,0,0,0,2004,90,1753,16610.00,5,0.0,1399,5094900.0,1,79


# Pisahkan Target dan Fitur

In [18]:
# Split fitur dan target
X = df.drop(columns=['Premi Asuransi', 'Kategori Risiko'])  # Fitur
y_reg = df['Premi Asuransi']  # Target regresi
y_cls = df['Kategori Risiko']  # Target klasifikasi

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train_reg, y_test_reg, y_train_cls, y_test_cls = train_test_split(
    X, y_reg, y_cls, test_size=0.2, random_state=42)

# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Normalisasi target regresi (Premi Asuransi)
scaler_y = StandardScaler()
y_train_reg = scaler_y.fit_transform(y_train_reg.values.reshape(-1, 1)).flatten()
y_test_reg = scaler_y.transform(y_test_reg.values.reshape(-1, 1)).flatten()

# Bangun Model TensorFlow

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt

# === Gunakan subset data untuk percepatan tuning ===
X_sample = X_train[:5000]
y_cls_sample = y_train_cls[:5000]
y_reg_sample = y_train_reg[:5000]

# === Fungsi model ===
def build_model(hp):
    inputs = layers.Input(shape=(X_train.shape[1],))

    x = layers.Dense(
        units=hp.Int('units', 64, 128, step=32),
        activation='relu'
    )(inputs)
    x = layers.Dropout(
        rate=hp.Float('dropout', 0.2, 0.4, step=0.1)
    )(x)

    risk_output = layers.Dense(3, activation='softmax', name='risk_output')(x)
    premium_output = layers.Dense(1, activation='linear', name='premium_output')(x)

    model = models.Model(inputs=inputs, outputs=[risk_output, premium_output])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('lr', 1e-4, 1e-3, sampling='log')
        ),
        loss={
            'risk_output': 'sparse_categorical_crossentropy',
            'premium_output': 'mse'
        },
        metrics={
            'risk_output': 'accuracy',
            'premium_output': 'mae'
        }
    )
    return model


In [21]:
from keras_tuner import Objective

# === Gunakan RandomSearch dulu untuk percepatan awal ===
tuner = kt.RandomSearch(
    build_model,
    objective=Objective("val_risk_output_accuracy", direction="max"),
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='asuransi_fast'
)

# === Callback ===
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# === Tuning ===
tuner.search(
    X_sample,
    {'risk_output': y_cls_sample, 'premium_output': y_reg_sample},
    validation_split=0.2,
    epochs=15,
    batch_size=128,
    callbacks=[early_stop]
)

Reloading Tuner from my_dir\asuransi_fast\tuner0.json


In [22]:
best_hp = tuner.get_best_hyperparameters(1)[0]
model = build_model(best_hp)

# Latih ulang dengan semua data dan lebih banyak epoch
model.fit(
    X_train,
    {'risk_output': y_train_cls, 'premium_output': y_train_reg},
    validation_split=0.2,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop]
)

Epoch 1/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.3095 - premium_output_loss: 0.8066 - premium_output_mae: 0.6169 - risk_output_accuracy: 0.8362 - risk_output_loss: 0.5029 - val_loss: 0.9208 - val_premium_output_loss: 0.6572 - val_premium_output_mae: 0.5260 - val_risk_output_accuracy: 0.8961 - val_risk_output_loss: 0.2636
Epoch 2/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9390 - premium_output_loss: 0.6575 - premium_output_mae: 0.5423 - risk_output_accuracy: 0.8899 - risk_output_loss: 0.2815 - val_loss: 0.8760 - val_premium_output_loss: 0.6509 - val_premium_output_mae: 0.5212 - val_risk_output_accuracy: 0.9123 - val_risk_output_loss: 0.2250
Epoch 3/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8980 - premium_output_loss: 0.6541 - premium_output_mae: 0.5323 - risk_output_accuracy: 0.9047 - risk_output_loss: 0.2439 - val_loss: 0.8593 - val_premium_output_loss: 0.6512 - val_premium_output_mae: 0.5099 - val_risk_output_accuracy: 0.9214 - val_risk_output_loss:

In [23]:
results = model.evaluate(
    X_train,
    {'risk_output': y_train_cls, 'premium_output': y_train_reg},
    batch_size=128,
    verbose=1
)

for name, value in zip(model.metrics_names, results):
    if 'risk_output' in name:
        print(f"[Klasifikasi Risiko] {name}: {value:.4f}")
    elif 'premium_output' in name:
        print(f"[Prediksi Premi]     {name}: {value:.4f}")
    else:
        print(f"{name}: {value:.4f}")

660/660 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7865 - premium_output_loss: 0.6038 - premium_output_mae: 0.4979 - risk_output_accuracy: 0.9282 - risk_output_loss: 0.1827
loss: 0.7817
compile_metrics: 0.1821
[Klasifikasi Risiko] risk_output_loss: 0.5994
[Prediksi Premi]     premium_output_loss: 0.4961


Artinya:

- loss: 0.7810 → total gabungan loss dari kedua output (klasifikasi + regresi)
- risk_output_loss: 0.5994 → loss khusus untuk output klasifikasi risiko
- premium_output_loss: 0.5007 → loss khusus untuk output regresi premi
- risk_output_accuracy: 92.78% → akurasi klasifikasi risiko sangat baik, sekitar 92.78%
- premium_output_mae: 0.5025 → rata-rata kesalahan absolut pada prediksi premi, sekitar 0.5 (nilai ini artinya prediksi premi rata-rata meleset ±0.5 dari nilai asli, tergantung skala target)

Kesimpulan singkat:

- Model klasifikasi risiko sudah sangat bagus, akurasinya > 90%
- Model prediksi premi juga cukup baik, dengan MAE sekitar 0.5

In [24]:
# Format .h5 (file tunggal)
model.save("ModelPremiKendaraan.h5")

In [25]:
import joblib

# Simpan scaler untuk fitur (X)
joblib.dump(scaler, 'scaler_Kendaraan_X.pkl')

# Simpan scaler untuk target regresi (y)
joblib.dump(scaler_y, 'scaler_Kendaraan_Y.pkl')


['scaler_Kendaraan_Y.pkl']